# 프롬프트 엔지니어링 기법
https://www.promptingguide.ai/kr/techniques

In [1]:
from dotenv import load_dotenv
import os
from flask import Flask, request, jsonify, render_template
# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")
# print(API_KEY)

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

### Zero-shot Prompting
: 예시가 주어지지 않음

In [2]:
prompt = '''
다음 영어 문장을 불어로 번역해줘?
'Hello, how are you?'
'''

completion = client.chat.completions.create(
    model = 'gpt-4o-mini',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)

'Hello, how are you?'는 프랑스어로 'Bonjour, comment ça va ?'입니다.


### Few-shot Prompting 
: 몇 개의 예시를 제공 , 한 개의 예시가 주어지는 경우 --> One-shot

In [3]:
prompt = """
English: 'Hello, how are you?'
French: 'Bonjour, comment ça va?'

English: 'What is your name?'
French: 'Comment vous appelez-vous?'

English: 'Hello,Miss'
French: 
"""

completion = client.chat.completions.create(
    model = 'gpt-4o-mini',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)

French: 'Bonjour, Mademoiselle'


In [4]:
prompt = """
아래 예시를 참조해서 알맞은 답변해줘

질문: '이 영화 너무 재미 없어!!'
답변: 부정 평가

질문: '이 영화 돈이 안 아까와..'
답변: 긍정 평가

질문: '시간만 버렸어~'
답변: 부정 평가

질문: '내내 하품만 나옴'
답변: 

질문: '핵 꿀잼~'
답변: 
"""

completion = client.chat.completions.create(
    model = 'gpt-4o-mini',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)

질문: '내내 하품만 나옴'  
답변: 부정 평가

질문: '핵 꿀잼~'  
답변: 긍정 평가


In [5]:
prompt = """
아래 예시를 참조해서 알맞은 답변해줘

질문: '이 영화 너무 재미 없어!!'
답변: 부정 평가

질문: '내내 하품만 나옴'
답변: 

질문: '핵 꿀잼~'
답변: 
"""

completion = client.chat.completions.create(
    model = 'gpt-4o-mini',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)

질문: '내내 하품만 나옴'  
답변: 부정 평가

질문: '핵 꿀잼~'  
답변: 긍정 평가


### CoT(Chain-of-Thought) Prompting

In [6]:
prompt = '''
철수에게는 사과가 10개 있었습니다 그저께 부터 오늘까지 매일 아침에 2개씩 먹었습니다
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다 오늘 저녁에 친구에게 2개를 주었습니다
철수는 총 몇 개의 사과를 가지고 있을까요?
'''
completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content) # 11개 or 9개: 틀린 답변

철수는 현재 11개의 사과를 가지고 있습니다. 처음에는 10개의 사과를 가졌고, 그저께와 어제 각각 2개씩 먹어서 6개가 줄었고, 어머니께서 5개를 더 사다 주셨으므로 6+5=11개가 되었습니다. 이후 친구에게 2개를 주었으므로 11-2=9개가 남게 됩니다.


In [7]:
prompt = '''
철수에게는 사과가 10개 있었습니다 그저께 부터 오늘까지 매일 아침에 2개씩 먹었습니다
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다 오늘 저녁에 친구에게 2개를 주었습니다
철수는 총 몇 개의 사과를 가지고 있을까요?
'''

completion = client.chat.completions.create(
    model = 'gpt-4o-mini',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)

철수의 사과 개수를 계산해 보겠습니다.

1. 처음에 철수는 사과가 10개 있었습니다.
2. 그저께부터 오늘까지 매일 아침에 2개씩 먹었습니다. 오늘이 포함된 3일 동안 사과를 먹었으므로:
   - 2개 × 3일 = 6개를 먹었습니다.
3. 따라서 오늘 아침까지 남은 사과는:
   - 10개 - 6개 = 4개입니다.
4. 오늘 점심에 어머니께서 사과 5개를 더 사다 주셨으므로:
   - 4개 + 5개 = 9개입니다.
5. 오늘 저녁에 친구에게 2개를 주었으므로:
   - 9개 - 2개 = 7개입니다.

결론적으로, 철수는 총 7개의 사과를 가지고 있습니다.


In [8]:
prompt = '''
철수에게는 사과가 10개 있었습니다 그저께 부터 오늘까지 매일 아침에 2개씩 먹었습니다
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다 오늘 저녁에 친구에게 2개를 주었습니다
철수는 총 몇 개의 사과를 가지고 있을까요? 단계별로 분리해서 생각해 보세요
'''
completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)  # 7개 : 정답(틀릴 경우도 있다, 9개)

1. 처음에 가지고 있던 사과의 수: 10개
2. 매일 아침에 먹은 사과의 수: 2개 x 2일 = 4개
3. 어머니가 사다 준 사과의 수: 5개
4. 친구에게 준 사과의 수: 2개

따라서, 철수가 가지고 있는 사과의 총 수는
10개 (처음에 가지고 있던 수) - 4개 (아침에 먹은 수) + 5개 (어머니가 사다 준 수) - 2개 (친구에게 준 수) = 9개

철수는 총 9개의 사과를 가지고 있습니다.


In [9]:
prompt="""
1. 철수에게는 사과가 10개 있었습니다.
2. 그저께부터 오늘까지 매일 아침에 2개씩 먹었습니다.
   (3일 동안 2개씩 먹어서 2 * 3 = 6개의 사과를 먹었습니다.)
3. 오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다.
4. 오늘 저녁에 친구에게 2개를 주었습니다.
철수는 총 몇 개의 사과를 가지고 있을까요?
"""

completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content) # 7개 : 정답

철수는 현재 7개의 사과를 가지고 있습니다. 처음에 10개의 사과가 있었고, 6개를 먹었으며, 어머니께서 5개를 더 사다 주셨고, 친구에게 2개를 주었기 때문에 10 - 6 + 5 - 2 = 7개의 사과가 남았습니다.


In [10]:
prompt='''
집합 {4, 8, 9, 15, 12, 2, 1}에서 홀수를 모두 더하면 짝수야.
답변: 홀수(9, 15, 1)를 모두 더하면 25가 돼. 위의 명제는 거짓이야.

집합 {17, 10, 19, 4, 8, 12, 24}에서 홀수를 모두 더하면 짝수야.
답변: 홀수(17, 19)를 모두 더하면 36이 돼. 위의 명제는 참이야.

집합 {16, 11, 14, 4, 8, 13, 24}에서 홀수를 모두 더하면 짝수야.
답변: 홀수(11, 13)를 모두 더하면 24가 돼. 위의 명제는 참이야.

집합 {17, 9, 10, 12, 13, 4, 2}에서 홀수를 모두 더하면 짝수야.
답변: 홀수(17, 9, 13)를 모두 더하면 39가 돼. 위의 명제는 거짓이야.

집합 {15, 32, 5, 13, 82, 7, 1}에서 홀수를 모두 더하면 짝수야.
답변:
'''
completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)  # 답변 : 거짓   (정답)

홀수(15, 5, 13, 7, 1)를 모두 더하면 41이 되어 위의 명제는 거짓입니다.


### SC(Self-Consistency) Prompting
: 여러번 질의해서 가장 빈도가 높은 답을 선택한다

In [14]:
from collections import Counter

# 동일한 질문에 대해 여러 번의 답변을 생성하여 Self-Consistency 유도
def generate_responses(prompt, n=5):
    responses = []
    for _ in range(n):
        response =  client.chat.completions.create(
            model="gpt-4o-mini",
            # model="gpt-3.5-turbo",
            messages=[ {"role": "user",  "content": prompt}],
            temperature=0.7,  # 다양성 확보를 위해 온도를 높게 설정
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        responses.append(response.choices[0].message.content)
    return responses

# 가장 자주 등장하는 답변을 선택
# response_count.most_common() 메서드는 Counter 객체에 저장된 요소들의 
# 빈도수를 기준으로 정렬된 리스트를 반환,  [0][0]는 첫번째 요소 선택
def select_most_consistent_response(responses):
    response_count = Counter(responses)
    # print(type(response_count))
    most_common_response = response_count.most_common(1)[0][0]
    # print(response_count.most_common(3))
    return most_common_response

# 문제 정의
prompt = """
철수에게는 사과가 10개 있었습니다. 그저께부터 오늘까지 매일 아침에 2개씩 먹었습니다.
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다.
오늘 저녁에 친구에게 2개를 주었습니다.
철수는 총 몇 개의 사과를 가지고 있을까요?
"""

# 여러 번 답변 생성
responses = generate_responses(prompt,n=5)

In [15]:
# 가장 일관성 있는 답변 선택
final_answer = select_most_consistent_response(responses)

# 결과 출력
print("Generated Responses:")
for i, response in enumerate(responses, 1):
    print(f"Response {i}: {response}")

print("\nMost Consistent Answer:")
print(final_answer)

# model="gpt-3.5-turbo" 경우에는 9개
# model="gpt-4o-mini" 경우에는 7개

Generated Responses:
Response 1: 철수가 처음에 가지고 있던 사과는 10개였습니다. 

그저께부터 오늘까지 매일 아침에 2개씩 먹었으므로, 
- 그저께 아침: 2개 
- 어제 아침: 2개 
- 오늘 아침: 2개 

총 3일 동안 매일 2개씩 먹었으므로, 
2개 × 3일 = 6개를 먹었습니다.

처음 사과의 개수에서 먹은 사과의 개수를 빼면:
10개 - 6개 = 4개

오늘 점심에 어머니께서 사과 5개를 더 사다 주셨으므로:
4개 + 5개 = 9개

그리고 오늘 저녁에 친구에게 2개를 주었으므로:
9개 - 2개 = 7개

따라서 철수가 현재 가지고 있는 사과는 총 7개입니다.
Response 2: 철수의 사과 개수를 계산해 보겠습니다.

1. 처음에 철수는 사과가 10개 있었습니다.
2. 그저께부터 오늘까지 매일 아침에 2개씩 먹었습니다. 
   - 그저께: 2개
   - 어제: 2개
   - 오늘: 2개
   - 총 3일 동안 2개씩 먹었으므로, 2개 × 3일 = 6개를 먹었습니다.
   
   철수가 남긴 사과의 수는:
   10개 - 6개 = 4개

3. 오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다.
   4개 + 5개 = 9개

4. 오늘 저녁에 친구에게 2개를 주었습니다.
   9개 - 2개 = 7개

따라서, 철수는 총 7개의 사과를 가지고 있습니다.
Response 3: 철수는 처음에 사과가 10개 있었습니다. 그저께부터 오늘까지 매일 아침에 2개씩 먹었습니다. 

- 그저께: 2개 먹음 (남은 사과: 10 - 2 = 8개)
- 어제: 2개 먹음 (남은 사과: 8 - 2 = 6개)
- 오늘 아침: 2개 먹음 (남은 사과: 6 - 2 = 4개)

오늘 점심에 어머니께서 사과 5개를 더 사다 주셨으므로, 

남은 사과: 4 + 5 = 9개

오늘 저녁에 친구에게 2개를 주었으므로,

남은 사과: 9 - 2 = 7개

따라서 철수는 총 7개의 사과를 가지고 있습니다.
Response 4: 철수는 처음에 사과가 10개 있

In [13]:
from collections import defaultdict
import re
prompt = """
4명의 생존자가 강을 건너야 합니다. 각 생존자가 강을 건너는 데 걸리는 시간은 각각 다릅니다: 3분, 4분, 7분, 12분.
이들은 한 번에 최대 두 명씩만 함께 배를 타고 강을 건널 수 있으며, 배에는 반드시 등불을 지참해야 합니다. 
그러나 등불은 하나뿐이어서, 생존자들이 강을 건널 때마다 등불을 반드시 함께 가져가야 합니다. 
모든 생존자가 가장 효율적으로 강을 건너려면 어떻게 해야 할까요? 
마지막으로 총 소요 시간을 '정답: <정답>분' 형식으로 답변하세요.
"""

def get_most_frequent_answer(prompt,iterations=10):
    answers = defaultdict(int)
    for idx in range(iterations):
        context = [{'role':'user', 'content':prompt}]
        response = client.chat.completions.create(
            # model = 'gpt-3.5-turbo',  # 다양한 답이 생성, 정답을 못 맞춤
            model = 'gpt-4o-mini',      # 정답이 거의 같음 : 27분, 빈도표: [('27분', 8), ('26분', 1), ('24분', 1)]
            messages = context,
            temperature = 0.2
        )
        response_content = response.choices[0].message.content

        print(f"\n{idx+1}번째 샘플:")
        print(response_content)
         
        match = re.search(r"정답: (\d+분)", response_content)
        
        if match:
            parsed_answer = match.group(1)
            answers[parsed_answer] += 1
        print('-'*60)  

    # 빈도가 가장 높은 답을 선택   
    sorted_answers = sorted(answers.items(),key=lambda x:x[1],reverse=True)
    print(f"\n빈도표: {sorted_answers}")
    most_frequent_answer = sorted_answers[0]
    return most_frequent_answer
  
most_frequent_answer = get_most_frequent_answer(prompt) 
print('최빈값:',most_frequent_answer)


1번째 샘플:
이 문제를 해결하기 위해서는 생존자들이 강을 건너는 순서를 잘 계획해야 합니다. 각 생존자의 강을 건너는 시간은 다음과 같습니다:

- A: 3분
- B: 4분
- C: 7분
- D: 12분

효율적으로 강을 건너기 위해 다음과 같은 순서를 따릅니다:

1. A와 B가 함께 강을 건넙니다. (4분 소요)
   - A와 B가 강을 건넌 후, B가 등불을 가지고 돌아옵니다. (4분 소요)
   
   총 소요 시간: 4 + 4 = 8분

2. C와 D가 함께 강을 건넙니다. (12분 소요)
   - C가 등불을 가지고 돌아옵니다. (7분 소요)
   
   총 소요 시간: 8 + 12 + 7 = 27분

3. 마지막으로 A와 B가 다시 함께 강을 건넙니다. (4분 소요)

최종적으로 모든 생존자가 강을 건너는 데 걸리는 총 시간은 27 + 4 = 31분입니다.

따라서, 정답은 다음과 같습니다:

정답: 31분
------------------------------------------------------------

2번째 샘플:
강을 건너는 최적의 방법은 다음과 같습니다:

1. 3분과 4분이 함께 강을 건넙니다. (4분 소요)
2. 3분이 돌아옵니다. (3분 소요)
3. 7분과 12분이 함께 강을 건넙니다. (12분 소요)
4. 4분이 돌아옵니다. (4분 소요)
5. 3분과 4분이 다시 함께 강을 건넙니다. (4분 소요)

이 과정을 통해 총 소요 시간은 다음과 같습니다:
4 + 3 + 12 + 4 + 4 = 27분

따라서, 정답은 다음과 같습니다:
정답: 27분
------------------------------------------------------------

3번째 샘플:
이 문제를 해결하기 위해서는 생존자들이 강을 건너는 최적의 순서를 찾아야 합니다. 각 생존자의 강을 건너는 시간은 다음과 같습니다:

- A: 3분
- B: 4분
- C: 7분
- D: 12분

효율적으로 강을 건너기 위해서는 가장 느린 생존자(D)를 최소

In [14]:
prompt = """
4명의 생존자가 강을 건너야 합니다. 각 생존자가 강을 건너는 데 걸리는 시간은 각각 다릅니다: 3분, 4분, 7분, 12분.
이들은 한 번에 최대 두 명씩만 함께 배를 타고 강을 건널 수 있으며, 배에는 반드시 등불을 지참해야 합니다. 
그러나 등불은 하나뿐이어서, 생존자들이 강을 건널 때마다 등불을 반드시 함께 가져가야 합니다. 
모든 생존자가 가장 효율적으로 강을 건너려면 어떻게 해야 할까요? 
마지막으로 총 소요 시간을 '정답: <정답>분' 형식으로 답변하세요.
"""

completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content) # 정답 못 맞춤

1. 3분과 4분이 함께 강을 건넙니다. (4분 소요)
2. 3분이 다시 돌아와 7분과 12분이 함께 강을 건넙니다. (12분 소요)
3. 4분이 다시 돌아와 3분과 4분이 함께 강을 건넙니다. (4분 소요)

따라서, 모든 생존자가 강을 건너는 데 총 20분이 소요됩니다.
정답: 20분


In [15]:
completion = client.chat.completions.create(
    model = 'gpt-4o-mini',
    temperature = 0.2,
    messages = [ {'role' : 'user',
                  'content' :  prompt}
               ]
)

print(completion.choices[0].message.content)  # 틀리는 경우 있음

이 문제를 해결하기 위해서는 생존자들이 강을 건너는 시간을 최소화하는 전략을 세워야 합니다. 각 생존자의 강을 건너는 시간은 다음과 같습니다:

- A: 3분
- B: 4분
- C: 7분
- D: 12분

효율적으로 강을 건너기 위해서는 빠른 생존자들이 돌아오는 시간을 최소화해야 합니다. 다음과 같은 단계로 진행할 수 있습니다:

1. A(3분)와 B(4분)가 함께 강을 건넙니다. (4분 소요)
   - 현재: C(7분), D(12분) / A, B 건너편

2. A(3분)가 돌아옵니다. (3분 소요)
   - 현재: A(3분), C(7분), D(12분) / B 건너편

3. C(7분)와 D(12분)가 함께 강을 건넙니다. (12분 소요)
   - 현재: A(3분) / B, C, D 건너편

4. B(4분)가 돌아옵니다. (4분 소요)
   - 현재: A(3분), B(4분) / C, D 건너편

5. A(3분)와 B(4분)가 다시 함께 강을 건넙니다. (4분 소요)
   - 현재: 모두 건너편

이 과정을 통해 총 소요 시간은 다음과 같습니다:

4 + 3 + 12 + 4 + 4 = 27분

따라서, 정답은 다음과 같습니다:

정답: 27분


In [16]:
# a = {'27분':5,'29분':1}
# a.items()

In [17]:
# import re
# data = '''
# 이 과정을 통해 모든 생존자가 강을 건너는 데 걸리는 총 시간은 39분입니다.
# 정답: 39분
# '''
# match = re.search(r"(정답): (\d+분)", data)
# parsed_answer = match.group(2)
# print(parsed_answer)

In [18]:
# data = [('a',30),('b',15),('c',20)]
# result = sorted(data,key=lambda x:x[1],reverse=True)
# print(result)

In [19]:
# def my_func(x):
#     return x[1]

# my_func(('a',30))

#### <br>
## Prompt Chaining (프롬프트 연쇄)
: 복잡한 문제를 해결하기 위해 여러 단계를 거쳐 연속적으로 프롬프트를 작성하는 방법 <br>
https://devocean.sk.com/blog/techBoardDetail.do?ID=166026&boardType=techBlog

In [34]:
def ask_openai(prompt,max_tokens=150):
    response = client.chat.completions.create(
        model = 'gpt-4o-mini', 
        max_tokens = max_tokens,
        messages = [ {'role' : 'user',
                     'content' :  prompt}
                   ]
    )
    return response.choices[0].message.content  
 
def product_development_chain():  # 제품 개발 체이닝

    # 1단계 : 제품 아이디어 구체화
    product_idea = '스마트 홈 보안 시스템'
    prompt_1 = f"{product_idea}의 핵심 기능과 그 기능들이 어떻게 사용자에게 가치를 제공할 수 있는지 설명해줘. 이 제품이 기존 시장에서 어떤 문제를 해결하는지에 대해 강조해줘."
    product_summary = ask_openai(prompt_1)
    print('-'*50)
    print('[1단계] : 제품 아이디어 구체화')
    print(f"제품 아이디어: {product_idea}\n제품 요약: {product_summary}\n")
 
    # 2단계 : 고객 요구사항 분석
    prompt_2 = f"{product_summary}을 바탕으로, 이 제품의 주요 타겟 고객이 어떤 요구사항을 가지고 있는지 분석해줘. 이 고객들이 제품을 사용하면서 겪을 수 있는 주요 문제와 기대되는 기능을 설명해줘." 
    customer_requirements = ask_openai(prompt_2 )
    print('-'*50)
    print('[2단계] : 고객 요구사항 분석')
    print(f"제품 요약: {product_summary}\n\n고객 요구사항 분석:\n{customer_requirements}\n")

    # 3단계 : 성공적인 출시 전략
    prompt_3 = f"{customer_requirements}을 기반으로, 이 제품을 성공적으로 출시하기 위한 전략을 제안해줘. 특히, 마케팅 전략과 고객 피드백을 반영하는 방법에 대해 자세히 설명해줘."
    launch_strategy = ask_openai(prompt_3)
    print('-'*50)
    print('[3단계] : 성공적인 출시 전략')
    print(f"고객 요구사항 분석:\n{customer_requirements}\n\n출시 전략:\n{launch_strategy}\n")

if __name__ == "__main__":
    product_development_chain()    

--------------------------------------------------
[1단계] : 제품 아이디어 구체화
제품 아이디어: 스마트 홈 보안 시스템
제품 요약: 스마트 홈 보안 시스템은 사용자에게 안전하고 편리한 생활을 제공하는 데 중요한 역할을 하는 기술입니다. 이러한 시스템의 핵심 기능과 그 기능들이 사용자에게 제공하는 가치는 다음과 같습니다:

### 1. 실시간 모니터링
- **기능 설명**: 카메라, 센서 등을 통해 집 안팎을 실시간으로 모니터링합니다.
- **가치 제공**: 사용자는 언제 어디서나 스마트폰이나 태블릿을 통해 자신의 집을 확인할 수 있습니다. 이는 불안감을 줄이고, 침입 감지 시 즉각적인 대응이 가능하게 합니다.

### 2. 자동 알림 및 경고 시스템
- **

--------------------------------------------------
[2단계] : 고객 요구사항 분석
제품 요약: 스마트 홈 보안 시스템은 사용자에게 안전하고 편리한 생활을 제공하는 데 중요한 역할을 하는 기술입니다. 이러한 시스템의 핵심 기능과 그 기능들이 사용자에게 제공하는 가치는 다음과 같습니다:

### 1. 실시간 모니터링
- **기능 설명**: 카메라, 센서 등을 통해 집 안팎을 실시간으로 모니터링합니다.
- **가치 제공**: 사용자는 언제 어디서나 스마트폰이나 태블릿을 통해 자신의 집을 확인할 수 있습니다. 이는 불안감을 줄이고, 침입 감지 시 즉각적인 대응이 가능하게 합니다.

### 2. 자동 알림 및 경고 시스템
- **

고객 요구사항 분석:
스마트 홈 보안 시스템의 주요 타겟 고객은 주로 가정 소유자, 특히 가족 구성원이 있는 가정과 혼자 사는 사람들입니다. 이 고객들이 가지고 있는 요구 사항과 기대되는 기능, 그리고 제품 사용 시 겪을 수 있는 문제를 다음과 같이 분석할 수 있습니다:

### 주요 타겟 고객의 요구 사항

1. **안전성**: 고객들은 자신의 집과 가족의 안전을 최우선으로 생각합니다. 이들은 

## Tree of Thoughts (ToT)
: **Tree of Thoughts (ToT)**는 대규모 언어 모델(LLM)이 복잡한 문제를 해결하기 위해 다양한 아이디어를 생성하고 평가하는 과정을 시뮬레이션하는 새로운 방법입니다. 마치 사람이 문제에 대한 해결책을 찾기 위해 마음속으로 여러 가지 가능성을 떠올리고 그중 가장 적절한 것을 선택하는 것과 유사합니다. <br>
### ToT가 작동하는 방식
생각의 나무(Tree of Thoughts): 문제를 해결하기 위한 다양한 아이디어와 접근 방식을 가지(branches)로 확장시키는 나무 구조로 표현합니다. 각 가지는 다양한 사고의 흐름을 나타냅니다.

탐색 과정: 나무의 뿌리(root)에서 시작하여 각 가지를 따라가며 가능한 해결 방법들을 평가하고, 최적의 해결책을 찾는 과정을 탐색이라고 합니다. 이 과정에서 여러 가지 경로를 비교하고, 실패하거나 비효율적인 경로는 가지치기(pruning)하여 제거합니다.

분기와 평가: 문제를 해결하는 다양한 경로를 탐색하는 중간 중간에 각 경로가 얼마나 유망한지 평가하고, 유망한 경로를 더 깊이 탐색하며 문제 해결에 접근합니다.]

![image](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2FTOT.3b13bc5e.png&w=1200&q=75)

In [46]:
def generate_thoughts(prompt,n=3):
    """
    GPT를 사용하여 주어진 프롬프트에 대해 여러 가지 생각(아이디어)을 생성하는 함수.
    """    
    response = client.chat.completions.create(
        model = 'gpt-4o-mini', 
        messages = [
            {'role' : 'system', 'content' : 'You are a helpful assistant.'},
            {'role' : 'user', 'content' :  prompt}
                   ],
        n = n,   # 생성할 생각(아이디어)의 수
        max_tokens=100        
    )
    thoughts = [choice.message.content for choice in response.choices]
    return thoughts


# 함수는 초기 생각들을 생성한 후, 각 생각에 대해 더 깊이 탐색하는 과정을 거칩니다.
# 더 깊이 탐색한 결과 중 길이가 가장 긴 응답을 최적의 생각으로 선택 
def  tree_of_thoughts(prompt,depth=2):
    """
    Tree of Thoughts 방식을 사용하여 최적의 생각을 찾는 함수.
    """ 
    initial_thoughts = generate_thoughts(prompt)
    print(f"Initial thoughts:\n{initial_thoughts}\n")

    best_thought = None
    for i, thought in enumerate(initial_thoughts):
        print(f"Exploring thought {i + 1} : {thought}")
        next_prompt = f"{thought} 다음은 어떻게 할 수 있을까요?"
        deeper_thoughts = generate_thoughts(next_prompt)

        # print(deeper_thoughts)
        
        # 간단한 평가 방법: 길이가 가장 긴 응답을 최선으로 선택
        best_deeper_thought = max(deeper_thoughts,key=len)
        print(f"Deeper thought : {best_deeper_thought}\n")
        
        # 최적의 생각 갱신
        if not best_thought or len(best_deeper_thought) > len(best_thought):
            best_thought = best_deeper_thought

    print(f"Best thought: {best_thought}")    

# 사용 예제
if __name__ == '__main__':
    user_prompt = "자연을 보존하기 위한 창의적인 아이디어가 무엇일까요?"
    tree_of_thoughts(user_prompt)

Initial thoughts:
['자연을 보존하기 위한 창의적인 아이디어는 여러 가지가 있습니다. 여기 몇 가지 제안을 드립니다:\n\n1. **도시 농업**: 도시 공간을 활용하여 유기농 채소와 과일을 재배하는 도시 농업 프로젝트를 추진합니다. 이를 통해 지역 식량 생산을 늘리고, 도시의 녹지 공간을 확장할 수 있습니다.\n\n2. **자연 기반 관광**: 자연을 보호하면서 지역 경제에 기', '자연을 보존하기 위한 창의적인 아이디어는 다양합니다. 몇 가지를 제안해 드릴게요.\n\n1. **커뮤니티 가드닝**: 지역 주민들이 함께 참여할 수 있는 커뮤니티 가든을 조성하여 식물 재배와 자연 보호를 촉진합니다. 이를 통해 유기농 식품을 생산하고, 지역 생태계를 유지하는 데 기여할 수 있습니다.\n\n2. **자연 교육 프로그램**', '자연을 보존하기 위한 창의적인 아이디어는 다양합니다. 여기 몇 가지를 제안드립니다:\n\n1. **도시 숲 조성**: 도시 내에 작은 숲이나 녹지를 조성하여 도시 생태계를 보호하고 시민들이 자연을 느낄 수 있는 공간을 제공합니다. 이를 통해 도시 열섬 효과를 줄이고 대기질을 개선할 수 있습니다.\n\n2. **커뮤니티 정원**: 지역 주민들이 함께']

Exploring thought 1 : 자연을 보존하기 위한 창의적인 아이디어는 여러 가지가 있습니다. 여기 몇 가지 제안을 드립니다:

1. **도시 농업**: 도시 공간을 활용하여 유기농 채소와 과일을 재배하는 도시 농업 프로젝트를 추진합니다. 이를 통해 지역 식량 생산을 늘리고, 도시의 녹지 공간을 확장할 수 있습니다.

2. **자연 기반 관광**: 자연을 보호하면서 지역 경제에 기
Deeper thought : 자연을 보존하기 위한 창의적인 아이디어에 이어서 구체적인 제안을 추가해보겠습니다:

3. **지역 주민 참여 프로그램**: 지역 주민들을 대상으로 자연 보호와 보존에 대한 교육 및 참여 프로그램을 실시합니다. 이 프로그램을 통해 주민들이 자연 환경의 중요성을 이해

#### <br>
## Retrieval Augmented Generation (RAG)
RAG의 기본 개념:
Retrieval (검색): 먼저, 특정 질문이나 문장에 대해 관련성이 높은 문서나 정보를 대규모 데이터베이스에서 검색합니다. 이 단계는 전통적인 정보 검색 시스템이나 특수한 인코더 모델(BERT와 같은 트랜스포머 기반 모델)을 사용하여 수행됩니다. 검색된 문서들은 후속 생성 과정의 입력으로 사용됩니다.

Augmentation (증강): 검색된 문서나 정보를 사용하여 입력된 질문이나 문장을 보강합니다. 즉, 검색된 관련 정보가 모델의 입력으로 주어지며, 이는 생성 모델이 더 나은 텍스트를 생성할 수 있도록 도와줍니다.

Generation (생성): 마지막으로, 검색된 정보를 바탕으로 주어진 질문에 대한 답변이나 설명을 생성합니다. 이 단계에서 GPT 생성 모델이 사용될 수 있으며, 검색된 정보가 포함되어 보다 정확하고 풍부한 응답을 생성할 수 있습니다.

In [6]:
def search_documents(query):
    # 이 함수는 문서를 검색하는 역할을 합니다.
    # 실제로는 외부 검색 엔진이나 데이터베이스에 쿼리를 보내야 하지만,
    # 이 예제에서는 간단히 미리 정의된 문서를 반환합니다.
    
    # 예제 문서
    documents = [
        "OpenAI는 인공지능 연구소로, 다양한 AI 기술을 개발합니다.",
        "GPT-4는 OpenAI에서 개발한 언어 모델로, 자연어 처리를 위한 강력한 도구입니다.",
        "RAG는 정보 검색과 텍스트 생성을 결합하여 더 나은 결과를 제공합니다."
    ]
    
    # 간단한 검색 논리: 입력 쿼리와 관련된 문서 필터링
    relevant_docs = [doc for doc in documents if query.split()[0] in doc]  # bug fix
    
    return relevant_docs

def generate_answer(query, documents):
    # 검색된 문서를 바탕으로 답변 생성
    combined_context = "\n".join(documents)
    prompt = f"질문: {query}\n\n문서:\n{combined_context}\n\n답변:"
    print(prompt)
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
                {"role": "user", 
                 "content": prompt}
                ],
        max_tokens=100,
        temperature=0.5
    )
    
    return response.choices[0].message.content

    

# query = "OpenAI는 무엇인가요?"
# query = "GPT-4는 무엇인가요?"
query = "RAG는 무엇인가요?"

# 1. 문서 검색
documents = search_documents(query)

# 2. 답변 생성
answer = generate_answer(query, documents)

print("질문:", query)
print("답변:", answer)   

질문: RAG는 무엇인가요?

문서:
RAG는 정보 검색과 텍스트 생성을 결합하여 더 나은 결과를 제공합니다.

답변:
질문: RAG는 무엇인가요?
답변: RAG는 "Retrieval-Augmented Generation"의 약자로, 정보 검색과 텍스트 생성을 결합한 기술입니다. 이 방법은 사용자가 질문을 했을 때, 관련된 정보를 검색하여 이를 바탕으로 더 정확하고 유용한 텍스트를 생성하는 방식입니다. RAG는 정보의 정확성과 생성된 텍스트의 품질을 동시에 향상시키는 데 도움을 줍니다.


In [59]:
# 문서 사전 참조 : 환각 현상 방지
system_role = """
신조어 사전:
{"스불재": "스스로 불러온 재앙의 줄임말로, 자신이 만든 문제로 곤란한 상황에 처했을 때 사용"
"억까": "억지로 까기의 줄임말로, 근거 없이 비판하거나 비난하는 상황을 말함"
"존맛탱": "존맛과 탱의 합성어로, 매우 맛있는 음식을 뜻함"
"어쩔티비": "상대방의 말을 무시하거나 별다른 대응을 하지 않겠다는 의미로, 주로 장난스럽게 사용된다"
"복세편살": "복잡한 세상 편하게 살자의 줄임말로, 복잡한 인생을 간단하게 살자는 의미"
"알잘딱깔센":"알아서 잘, 딱, 깔끔하고 센스 있게의 줄임말로, 어떤 일을 척척 잘 해내는 사람을 칭찬할 때 쓰임"}

이 신조어 사전을 참조하여 답변하세요. 만일 이 신조어 사전에 없으면 "모르는 단어입니다"라고 답하세요."
"""

# "점메추": "점심 메뉴 추천의 줄임말로, 점심에 무엇을 먹을지 고민할 때 사용"
# "킹받다": "열받다의 변형 표현으로, 아주 화가 나거나 짜증이 날 때 사용"
# "갓생": "신처럼 성실하고 부지런하게 사는 삶을 의미하며, 목표를 향해 최선을 다하는 삶을 칭찬할 때 사용"

prompt = "신조어 {신조어}의 뜻을 말해주세요.".format(신조어="스불재")
context = [#{"role": "system", "content": system_role},  
           {"role": "user", "content": prompt}] 
response = client.chat.completions.create(
            # model="gpt-4o-mini", 
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
            top_p=0
        )
print(response.choices[0].message.content)
# 환각(Hallucination) 현상 발생

"스불재"는 "스벅에서 불러주는 재료"를 줄인 말로, 스타벅스에서 주문 시 사용되는 용어나 단어를 의미합니다. 이는 스타벅스에서 일하는 직원들이 서로 소통하거나 주문을 처리할 때 사용하는 특별한 용어들을 가리키는 신조어입니다.


In [63]:
prompt = "신조어 {신조어}의 뜻을 말해주세요.".format(신조어="스불재")
context = [{"role": "system", "content": system_role},
           {"role": "user", "content": prompt}] 
response = client.chat.completions.create(
            # model="gpt-4o-mini", 
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
            top_p=0
        )
print(response.choices[0].message.content)
# 검색 증강으로 올바른 답 출력

"스불재"는 '스스로 불러온 재앙'을 줄인 말로, 자신이 만든 문제로 곤란한 상황에 처했을 때 사용하는 표현입니다.


In [69]:
prompt = "신조어 {신조어}의 뜻을 말해주세요.".format(신조어="스불재")
context = [#{"role": "system", "content": system_role},
           {"role": "user", "content": prompt}] 
response = client.chat.completions.create(
            model="gpt-4o-mini", 
            # model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
            top_p=0
        )
print(response.choices[0].message.content)

"스불재"는 "스스로 불행해지는 재능"의 줄임말로, 자신이 스스로 불행한 상황을 만들어내는 행동이나 성향을 의미합니다. 주로 사람의 선택이나 행동이 결과적으로 자신에게 불행을 초래하는 경우에 사용됩니다. 이 표현은 주로 SNS나 온라인 커뮤니티에서 유행하며, 자기 비하적인 맥락에서 사용되기도 합니다.


In [73]:
prompt = "신조어 {신조어}의 뜻을 말해주세요.".format(신조어="어쩔티비")
context = [{"role": "system", "content": system_role},
           {"role": "user", "content": prompt}] 
response = client.chat.completions.create(
            model="gpt-4o-mini", 
            # model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
            top_p=0
        )
print(response.choices[0].message.content)

"어쩔티비"는 상대방의 말을 무시하거나 별다른 대응을 하지 않겠다는 의미로, 주로 장난스럽게 사용됩니다.


## ReAct(Reasoning & Acting) 
: 논리적 추론(Reasoning)과 행동(Acting)을 결합하여 더 복잡하고 정교한 응답을 생성하도록 하는 프롬프트 설계 기법

In [77]:
# 간단한 영화 데이터베이스 예시
movie_database = {
    "액션": ["매드 맥스: 분노의 도로", "다크 나이트", "글래디에이터"],
    "코미디": ["슈퍼배드", "덤 앤 더머", "미스터 빈"],
    "로맨스": ["타이타닉", "노트북", "라라랜드"],
    "SF": ["인터스텔라", "인셉션", "매트릭스"]
}

def recommend_movies(genre):
    """
    장르를 입력받아 해당 장르의 영화를 추천하는 함수.
    """
    return movie_database.get(genre, [])

def react_prompt(query):
    """
    사용자의 질의를 바탕으로 추론하고, 영화 추천을 수행하는 ReAct 패턴 함수.
    """
    # 1. 모델에게 문제를 설명하고 추론을 요구하는 프롬프트 생성
    prompt = f"""
    문제: {query}

    단계 1: 사용자의 선호도를 분석하고 필요한 정보를 식별하세요.
    단계 2: 필요한 정보를 가져와 문제를 해결하세요.
    단계 3: 최종 영화 추천을 제공하세요.
    
    추론:
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
                {"role": "user", 
                 "content": prompt}
                ],
        max_tokens=150,
        temperature=0.5
    )

    reasoning = response.choices[0].message.content

    # 2. 추론을 통해 필요한 행동을 결정
    if "액션" in query:
        genre = "액션"
    elif "코미디" in query:
        genre = "코미디"
    elif "로맨스" in query:
        genre = "로맨스"
    elif "SF" in query:
        genre = "SF"
    else:
        genre = None
    
    if genre:
        action_result = recommend_movies(genre)
        action_result_text = f"{genre} 장르의 추천 영화는: {', '.join(action_result)}입니다."
    else:
        action_result_text = "추천할 영화 장르를 찾을 수 없습니다."

    # 3. 최종 결과를 생성하는 프롬프트
    final_prompt = f"""
    문제: {query}

    추론: {reasoning}

    행동 결과: {action_result_text}

    최종 답변:
    """

    final_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
                {"role": "user", 
                 "content": final_prompt}
                ],
        #max_tokens=200,
        temperature=0.5
    )

    return final_response.choices[0].message.content

def main():
    query = "액션 영화를 추천해줘"

    # ReAct 패턴을 사용하여 답변 생성
    answer = react_prompt(query)
    
    print("질문:", query)
    print("답변:", answer)

if __name__ == "__main__":
    main()

질문: 액션 영화를 추천해줘
답변: 액션 영화를 추천해드리기 위해 몇 가지 질문을 드리고 싶습니다. 아래의 정보를 알려주시면 더욱 맞춤형 추천을 해드릴 수 있습니다.

1. 선호하는 액션 영화의 스타일은 무엇인가요? (예: 슈퍼히어로, 스파이, 전쟁, 범죄 등)
2. 선호하는 영화의 시대는 어떻게 되나요? (예: 최신, 클래식, 특정 년대 등)
3. 어떤 분위기의 영화를 선호하시나요? (예: 어두운, 유머가 있는, 현실적인 등)
4. 특별히 좋아하는 배우나 감독이 있나요? 아니면 시리즈물에 관심이 있으신가요?

이 정보를 바탕으로 추천을 드리겠습니다!
